In [ ]:
# Import modules
import subprocess
from pathlib import Path

from hydroflows.log import setuplog
from hydroflows.methods import rainfall, sfincs
from hydroflows.utils.example_data import fetch_data
from hydroflows.workflow import Workflow, WorkflowConfig

logger = setuplog(level="INFO")

In [ ]:
# Fetch the build data
cache_dir = fetch_data(data="global-data")

In [ ]:
# Define case name and root directory
name = "pluvial_multiple_regions"
pwd = Path().resolve()  # Get the current file location
case_root = Path(pwd, "cases", name)  # output directory

In [ ]:
# Setup the configuration
config = WorkflowConfig(
    catalog_path=Path(cache_dir, "data_catalog.yml"),
    sfincs_run_method = "docker",
    start_date="2014-01-01",
    end_date="2021-12-31",
    # sfincs settings
    hydromt_sfincs_config=Path(pwd, "hydromt_config/sfincs_config.yml"),
    subgrid_output=True,
    # design event settings
    rps=[2, 5, 10],
)

In [ ]:
# Setup the workflow
w = Workflow(
    config=config,
    wildcards={"region": ["region", "region2"]},
    name=name,
    root=case_root,
)

In [ ]:
# Build the SFINCS models
sfincs_build = sfincs.SfincsBuild(
    region="../../data/build/{region}.geojson",  # NOTE: case in sub-subfolder of pwd
    sfincs_root="models/sfincs/{region}",
    config=w.get_ref("$config.hydromt_sfincs_config"),
    catalog_path=w.get_ref("$config.catalog_path"),
    subgrid_output=w.get_ref("$config.subgrid_output"),
    )
w.create_rule(sfincs_build, "sfincs_build")


In [ ]:
# Get Rainfall timeseries
get_rainfall = rainfall.GetERA5Rainfall(
    region=sfincs_build.output.sfincs_region,
    data_root="data/era5/{region}",
    start_date=w.get_ref("$config.start_date"),
    end_date=w.get_ref("$config.end_date"),
)
w.create_rule(get_rainfall, rule_id="get_rainfall")


In [ ]:
# Derive pluvial events from rainfall data
pluvial_events = rainfall.PluvialDesignEvents(
    precip_nc=get_rainfall.output.precip_nc,
    event_root="data/events/{region}_events",
    rps=w.get_ref("$config.rps"),
    wildcard="pluvial_events",
)
w.create_rule(pluvial_events, rule_id="pluvial_events")


In [ ]:
# Update the SFINCS models
sfincs_update = sfincs.SfincsUpdateForcing(
    sfincs_inp=sfincs_build.output.sfincs_inp,
    event_yaml=pluvial_events.output.event_yaml,
    event_name="{pluvial_events}",
    output_dir=sfincs_build.output.sfincs_inp.parent/"simulations"/"{pluvial_events}",
    copy_model=True # Necessary for CWL to work
)
w.create_rule(sfincs_update, rule_id="sfincs_update")


In [ ]:
# simulate and downscale pluvial flood maps

# Run SFINCS model
sfincs_run = sfincs.SfincsRun(
    sfincs_inp=sfincs_update.output.sfincs_out_inp,
    run_method=w.get_ref("$config.sfincs_run_method"),
)
w.create_rule(sfincs_run, rule_id="sfincs_run")

# Post process the results from pluvial events
sfincs_post = sfincs.SfincsDownscale(
    sfincs_map=sfincs_run.output.sfincs_map,
    sfincs_subgrid_dep=sfincs_build.output.sfincs_subgrid_dep,
    output_root="output/{region}",
)
w.create_rule(sfincs_post, "sfincs_post")


In [ ]:
# Do a dry run of the workflow
w.dryrun()

In [ ]:
w.to_cwl(cwlfile=w.root/"workflow.cwl", dryrun=False)

In [ ]:
# Write the workflow to a Snakefile and snakefile.config.yml
w.to_snakemake()

# show the top 25 lines of the Snakefile
with open(w.root / "Snakefile", "r") as f:
    for _ in range(25):
        print(f.readline().strip('\n'))

In [ ]:
from IPython.display import SVG

# (test) run the workflow with snakemake and visualize the directed acyclic graph
# make sure to have snakemake installed in your environment
subprocess.run('snakemake --dag | dot -Tsvg > dag.svg', cwd=w.root, shell=True).check_returncode()

# show the dag
SVG(Path(w.root, "dag.svg").as_posix())

In [ ]:
# uncomment to run the workflow
# subprocess.run(["snakemake", "-c", "1"], cwd=w.root)